In [103]:
include("src/GalacticDynamo.jl")

Main.GalacticDynamo

In [104]:
using Plots
using Measures
using ProgressMeter
using Statistics

In [105]:
# Constants
PARSEC_TO_METER = 3.086e+16
YEAR_TO_SECOND = 3.154e+7
 

3.154e7

In [106]:
h_0 = 0.5 # kpc
h_0 = h_0 * 1e3 * PARSEC_TO_METER
v_rms = 10 * 1e3 # m/s
τ = 10 # Myr
τ = τ * 1e6 * YEAR_TO_SECOND
η = 1/3 * τ * v_rms^2



unit_time = h_0^2 / η
unit_length = h_0
unit_field = 10^(-6) # Gauss

τ = τ / unit_time
v_rms = v_rms / (unit_length / unit_time)
η = η / (unit_length^2 / unit_time)
h_0 = h_0 / unit_length

r_min = 0.1 * 1e3 * PARSEC_TO_METER  / unit_length # kpc
r_max = 10.0 * 1e3 * PARSEC_TO_METER / unit_length # kpc

t_min = 0.0
t_max = 40 * 1e9 * YEAR_TO_SECOND / unit_time
n_t = 60000
d_t = (t_max - t_min) / n_t

t = LinRange(t_min, t_max, n_t)
dt = t[2] - t[1]

r_h = r_max

n_r = 100
d_r = (r_max - r_min) / n_r

n_ghost_zones = 5
n_proper_zones = n_r + n_ghost_zones


r_min_ghost = r_min - n_ghost_zones * d_r
r_max_ghost = r_max + n_ghost_zones * d_r
nr_ghost = n_r + 2 * n_ghost_zones

println("r_min_ghost = ", r_min_ghost)
println("r_max_ghost = ", r_max_ghost)
println("nr_ghost = ", nr_ghost)

r = LinRange(r_min_ghost, r_max_ghost, nr_ghost)
println(collect(r))
dr = r[2] - r[1]

Ω_0 = 170 / PARSEC_TO_METER * unit_time # s^-1
r_Ω = 2.0 * 1e3 * PARSEC_TO_METER / unit_length


r_min_ghost = -0.79
r_max_ghost = 20.99
nr_ghost = 110
[-0.79, -0.5901834862385322, -0.3903669724770643, -0.19055045871559628, 0.009266055045871524, 0.20908256880733933, 0.40889908256880736, 0.6087155963302752, 0.808532110091743, 1.008348623853211, 1.208165137614679, 1.4079816513761467, 1.6077981651376148, 1.8076146788990828, 2.00743119266055, 2.2072477064220184, 2.407064220183486, 2.606880733944954, 2.806697247706422, 3.00651376146789, 3.206330275229358, 3.4061467889908252, 3.6059633027522935, 3.8057798165137613, 4.00559633027523, 4.205412844036697, 4.405229357798166, 4.605045871559633, 4.8048623853211, 5.004678899082569, 5.204495412844036, 5.404311926605505, 5.604128440366972, 5.803944954128441, 6.003761467889908, 6.203577981651376, 6.403394495412844, 6.603211009174311, 6.80302752293578, 7.0028440366972475, 7.202660550458716, 7.402477064220183, 7.6022935779816505, 7.802110091743119, 8.001926605504588, 8.201743119266053, 8.401559633027523, 8.60137614678899, 8.80119266055046, 9.0010091

4.0

In [107]:
h = h_0 * ones(length(r))
Ω = Ω_0 ./ sqrt.(1 .+ (r ./ r_Ω) .^ 2)
q = -(r / Ω) * GalacticDynamo.gradient(Ω, r)

α = (τ^2 .* v_rms .^ 2 .* Ω) ./ h

C = 2 * η * d_t / d_r^2
V_z_fact = 0.0
V_r = zeros(length(r))
V_ϕ = zeros(length(r))
V_z = V_z_fact * ones(length(r)) / (unit_length / unit_time)




println("The correlation length (in kpc) is $(v_rms * τ / 1e3 / PARSEC_TO_METER)")
println("Diffusivity, η (in pc^2/Myr) is $(η / PARSEC_TO_METER^2 * YEAR_TO_SECOND * 1e6)")
println("Turbulent Diffusion time (in Myr) is $(h_0^2 / η / 1e6 / YEAR_TO_SECOND)")
println("Length step, dr (in kpc) is $(dr * unit_length / 1e3 / PARSEC_TO_METER)")
println("Time step, dt (in Myr) is $(dt * unit_time / 1e6 / YEAR_TO_SECOND)")
println("Courant limit, C is $C")
println("Velocity limit is $(d_r * unit_length / d_t / unit_time / 1e3)")


The correlation length (in kpc) is 6.623687600515615e-21
Diffusivity, η (in pc^2/Myr) is 3.3118438002578076e-20
Turbulent Diffusion time (in Myr) is 3.170577045022194e-14
Length step, dr (in kpc) is 0.09990825688073392
Time step, dt (in Myr) is 0.6666777779629661
Courant limit, C is 0.04736729969056727
Velocity limit is 145.2983512999366


In [108]:
# Radial profiles of α and Ω
plot(r * unit_length / 1e3 / PARSEC_TO_METER, α / unit_time,
    xlabel="r [kpc]", ylabel="α [kpc/s]", label="Radial profile of α", 
    lw=2, legend=:topright, size=(800, 600),
    dpi=1200)

plot!(margin = 5mm)
savefig("plots/alpha_profile.png",)


┌ Warning: No strict ticks found
└ @ PlotUtils /home/kmaitreys/.julia/packages/PlotUtils/8mrSm/src/ticks.jl:194
┌ Warning: No strict ticks found
└ @ PlotUtils /home/kmaitreys/.julia/packages/PlotUtils/8mrSm/src/ticks.jl:194


"/home/kmaitreys/Documents/college/10-2024-spring/p464-magnetohydrodynamics/project/plots/alpha_profile.png"

In [109]:
plot(r * unit_length / 1e3 / PARSEC_TO_METER, Ω / unit_time,
    xlabel="r [kpc]", ylabel="Ω [kpc/s]", label="Radial profile of Ω", 
    lw=2, legend=:topright, size=(800, 600),
    dpi=1200)

plot!(margin = 5mm)

savefig("plots/omega_profile.png")

"/home/kmaitreys/Documents/college/10-2024-spring/p464-magnetohydrodynamics/project/plots/omega_profile.png"

In [118]:
# Radial profiles of Dynamo number
# println("alpha = $α")
# println("q = $q")
# println("Ω = $Ω")
# println("h = $h")
# println("η = $η")


D = -α .* q .* Ω .* h.^3 / η.^2

println("Dynamo number is $(mean(D))")


plot(r * unit_length / 1e3 / PARSEC_TO_METER, D,
    xlabel="r [kpc]", ylabel="D", label="Radial profile of 
Dynamo number", 
    lw=2, legend=:topright, size=(800, 600),
    dpi=1200)

plot!(margin = 5mm)

savefig("plots/dynamo_number_profile.png")

Dynamo number is -56.2409823915437


"/home/kmaitreys/Documents/college/10-2024-spring/p464-magnetohydrodynamics/project/plots/dynamo_number_profile.png"

In [111]:
# print(collect(r))

B_r = (
    1 .* 
    sin.((π.*(r_min .- r)/(r_max-r_min))) .*
    exp.(-r.^2/r_Ω^2)
)
B_ϕ = (
    - 1 .*
    sin.((π.*(r_min .- r)/(r_max-r_min))) .*
    exp.(-r.^2/r_Ω^2)
)

println(GalacticDynamo.gradient(B_r, r)) 


[-0.1406068500747007, -0.14463821383388098, -0.15155723932187315, -0.15611708457980744, -0.15820597663333952, -0.15777710926167232, -0.15485023479465787, -0.14951080051716048, -0.14190667412039584, -0.13224261604320506, -0.12077276075454398, -0.10779145817065278, -0.0936228956232615, -0.07860996657437962, -0.06310287262179122, -0.04744793990083082, -0.03197710100922104, -0.016998441771710737, -0.002788142450547546, 0.010415939768563778, 0.022418890654275825, 0.03307149744542277, 0.04227117177364776, 0.04996114436307529, 0.0561281086590062, 0.06079854559320029, 0.06403400024376031, 0.06592560256115386, 0.06658812906148166, 0.06615389183952469, 0.06476671760039628, 0.06257624537023701, 0.05973273015691014, 0.05638249420782893, 0.05266412064476525, 0.04870543883449523, 0.04462130913520374, 0.04051217837758003, 0.036463347773433324, 0.03254487251586293, 0.02881199725278681, 0.025306023546090798, 0.022055503684092353, 0.019077658853029696, 0.01637992759722443, 0.013961561550573823, 0.011815

In [112]:
println(collect(r[n_ghost_zones+1:end]))

[0.20908256880733933, 0.40889908256880725, 0.608715596330275, 0.808532110091743, 1.008348623853211, 1.208165137614679, 1.4079816513761467, 1.6077981651376145, 1.8076146788990823, 2.00743119266055, 2.2072477064220184, 2.4070642201834858, 2.606880733944954, 2.806697247706422, 3.0065137614678896, 3.2063302752293574, 3.4061467889908257, 3.6059633027522935, 3.805779816513761, 4.005596330275229, 4.205412844036697, 4.405229357798166, 4.605045871559632, 4.8048623853211, 5.004678899082569, 5.204495412844036, 5.404311926605504, 5.604128440366972, 5.80394495412844, 6.003761467889908, 6.203577981651375, 6.403394495412843, 6.603211009174312, 6.803027522935778, 7.0028440366972475, 7.202660550458716, 7.402477064220183, 7.602293577981651, 7.802110091743118, 8.001926605504586, 8.201743119266055, 8.401559633027523, 8.601376146788992, 8.801192660550457, 9.001009174311925, 9.200825688073394, 9.400642201834861, 9.600458715596329, 9.800275229357798, 10.000091743119265, 10.199908256880732, 10.3997247706422, 

In [113]:
# initialize figure size
# plot(size=(8, 5))

# X and Y labels
# plot("R [kpc]", "B [microG]", label="B_r", legend=:topleft)
# plot B_r
plot(r[n_ghost_zones+1:end], B_r[n_ghost_zones+1:end], label="B_r (initial)",
xlabel = "Radius [kpc]", ylabel = "B [microG]", lw=2)
# plot B_ϕ
plot!(r[n_ghost_zones+1:end], B_ϕ[n_ghost_zones+1:end], label="B_ϕ (initial)",
xlabel = "Radius [kpc]", ylabel = "B [microG]",
lw=2)
plot!(legend=:bottomright, grid=true, size=(800, 500))
plot!(margin = 5mm)
plot!(dpi=1200)
# Save figure
savefig("plots/dynamo.png")
# show plot
# display(plot!)

"/home/kmaitreys/Documents/college/10-2024-spring/p464-magnetohydrodynamics/project/plots/dynamo.png"

In [138]:
# Integration
# print(dt)
evolution_B_r = zeros(length(r), n_t)
evolution_B_ϕ = zeros(length(r), n_t)

# size(evolution_B_r[1, :])
for i in 1:n_t
    time_step = dt
    B_r, B_ϕ = GalacticDynamo.runge_kutta_with_ghost_zones(
        B_r, B_ϕ, dt, r, η,V_z, h, α, Ω, q,n_proper_zones, n_ghost_zones
    )
    evolution_B_r[:, i] .= B_r
    evolution_B_ϕ[:, i] .= B_ϕ
end




# Create heatmap for Br
heatmap(t, r[1:end], evolution_B_r', aspect_ratio=:equal, xlabel="r", ylabel="Time Step", title="Evolution of Br")
savefig("plots/evolution_B_r.png")
# Create heatmap for Bϕ
heatmap(t, r[1:end], evolution_B_ϕ', aspect_ratio=:equal, xlabel="r", ylabel="Time Step", title="Evolution of Bϕ")

savefig("plots/evolution_B_ϕ.png")

ArgumentError: ArgumentError: Length of x & y does not match the size of z.
Must be either `size(z) == (length(y), length(x))` (x & y define midpoints)
or `size(z) == (length(y)+1, length(x)+1))` (x & y define edges).
